In [15]:
import numpy as np
import pandas as pd
import sys
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [16]:
import torch
import torch.nn as nn
from torchvision.datasets import CIFAR10
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torch.nn.functional as F

In [17]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/models")

In [18]:
from Classical_ML_Test import CML_Test
from format_data_cnn import Restructure

## Generate Data Matrices

In [19]:
healthy_name_list = ["K001", "K002", "K003", "K004", "K005", "K006"]
healthy_df_list = []
for k_name in healthy_name_list:
    for i in range(4):
        df = pd.read_csv("../data/processed/paderborn/down_sampled/" + k_name + "_K" + str(i) + "_down.csv", engine='c', low_memory=False, memory_map=True)
        del df["temp"]
        print("Start convert", k_name, "- K", i)
        healthy_df_list.append(Restructure(df).to_windows(1600))
healthy_df = np.concatenate(healthy_df_list)
healthy_label = np.repeat(0, healthy_df.shape[0])

Start convert K001 - K 0


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 6061.13it/s]


Start convert K001 - K 1


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 6895.97it/s]


Start convert K001 - K 2


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 11750.56it/s]


Start convert K006 - K 2


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 10519.03it/s]


Start convert K006 - K 3


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 10526.81it/s]


In [20]:
ir_name_list = ["KA01", "KA03", "KA05", "KA06", "KA07", "KA08", "KA09", "KA04", "KA15", "KA16", "KA22", "KA30"]
ir_df_list = []
for k_name in ir_name_list:
    for i in range(4):
        df = pd.read_csv("../data/processed/paderborn/down_sampled/" + k_name + "_K" + str(i) + "_down.csv", engine='c', low_memory=False, memory_map=True)
        del df["temp"]
        print("Start convert", k_name, "- K", i)
        ir_df_list.append(Restructure(df).to_windows(1600))
ir_df = np.concatenate(ir_df_list)
ir_label = np.repeat(1, ir_df.shape[0])

Start convert KA01 - K 0


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 15395.05it/s]


Start convert KA01 - K 1


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 16669.53it/s]


Start convert KA01 - K 2


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 14288.45it/s]


Start convert KA30 - K 2


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 14293.57it/s]


Start convert KA30 - K 3


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 15386.01it/s]


In [21]:
or_name_list = ["KI01", "KI03", "KI04", "KI05", "KI07", "KI08", "KI14", "KI16", "KI17", "KI18", "KI21"]
# "KB23", "KB24", "KB27" are IR + OR
or_df_list = []
for k_name in or_name_list:
    for i in range(4):
        df = pd.read_csv("../data/processed/paderborn/down_sampled/" + k_name + "_K" + str(i) + "_down.csv", engine='c', low_memory=False, memory_map=True)
        del df["temp"]
        print("Start convert", k_name, "- K", i)
        or_df_list.append(Restructure(df).to_windows(1600))
or_df = np.concatenate(or_df_list)
or_label = np.repeat(2, or_df.shape[0])

Start convert KI01 - K 0


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 13333.45it/s]


Start convert KI01 - K 1


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 15375.86it/s]


Start convert KI01 - K 2


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 197/197 [00:00<00:00, 11593.30it/s]


Start convert KI21 - K 2


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 10006.21it/s]


Start convert KI21 - K 3


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 9087.14it/s]


In [31]:
X = np.concatenate((healthy_df, ir_df, or_df))
y = np.concatenate((healthy_label, ir_label, or_label)) 

## Split data and create torch datasets

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [37]:
X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)
y_train= torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

In [38]:
import torch.utils.data as data_utils
batch_size = 128 
train_tensor = data_utils.TensorDataset(X_train, y_train) 
train_loader = data_utils.DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = True)

In [39]:
batch_size = 1024
test_tensor = data_utils.TensorDataset(X_test, y_test) 
test_loader = data_utils.DataLoader(dataset = test_tensor, batch_size = batch_size, shuffle = False)

In [40]:
X_train.size()

torch.Size([18453, 6, 40, 40])

In [41]:
X_test.size()

torch.Size([4614, 6, 40, 40])

In [42]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(6, 32, kernel_size=4,stride=1,padding = 1)
        self.mp1 = nn.MaxPool2d(kernel_size=4,stride=2)
        self.conv2 = nn.Conv2d(32,64, kernel_size=4,stride =1)
        self.mp2 = nn.MaxPool2d(kernel_size=4,stride=2)
        self.fc1= nn.Linear(2304,256)
        self.dp1 = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(256,3)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp1(self.conv1(x)))
        x = F.relu(self.mp2(self.conv2(x)))
        x = x.view(in_size,-1)
        x = F.relu(self.fc1(x))
        x = self.dp1(x)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)

In [43]:
cnn = CNN().double()

In [44]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)

In [45]:
num_epochs = 100

In [47]:
total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (signals, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        # Run the forward pass
        signals=signals
        labels=labels
        outputs = cnn(signals.double())
        loss = criterion(outputs, labels.long())
        
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        
        loss.backward()
        optimizer.step()
        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels.long()).sum().item()
        acc_list.append(correct / total)

        if (epoch+1) % 5 == 0 or epoch==0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Train Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))
        

Epoch [1/100], Step [1/145], Loss: 1.0814, Train Accuracy: 42.19%
Epoch [1/100], Step [2/145], Loss: 1.0718, Train Accuracy: 46.88%
Epoch [1/100], Step [3/145], Loss: 1.0887, Train Accuracy: 33.59%
Epoch [1/100], Step [4/145], Loss: 1.0712, Train Accuracy: 50.78%
Epoch [1/100], Step [5/145], Loss: 1.1408, Train Accuracy: 39.84%
Epoch [1/100], Step [6/145], Loss: 1.0908, Train Accuracy: 37.50%
Epoch [1/100], Step [7/145], Loss: 1.0779, Train Accuracy: 39.84%
Epoch [1/100], Step [8/145], Loss: 1.0708, Train Accuracy: 43.75%
Epoch [1/100], Step [9/145], Loss: 1.0782, Train Accuracy: 38.28%
Epoch [1/100], Step [10/145], Loss: 1.0851, Train Accuracy: 40.62%
Epoch [1/100], Step [11/145], Loss: 1.0802, Train Accuracy: 38.28%
Epoch [1/100], Step [12/145], Loss: 1.0799, Train Accuracy: 40.62%
Epoch [1/100], Step [13/145], Loss: 1.0773, Train Accuracy: 39.84%
Epoch [1/100], Step [14/145], Loss: 1.0621, Train Accuracy: 46.09%
Epoch [1/100], Step [15/145], Loss: 1.0761, Train Accuracy: 43.75%
Epoc

Epoch [1/100], Step [124/145], Loss: 1.0440, Train Accuracy: 42.97%
Epoch [1/100], Step [125/145], Loss: 1.0705, Train Accuracy: 40.62%
Epoch [1/100], Step [126/145], Loss: 1.0465, Train Accuracy: 41.41%
Epoch [1/100], Step [127/145], Loss: 1.0404, Train Accuracy: 41.41%
Epoch [1/100], Step [128/145], Loss: 1.0022, Train Accuracy: 50.00%
Epoch [1/100], Step [129/145], Loss: 1.0475, Train Accuracy: 49.22%
Epoch [1/100], Step [130/145], Loss: 1.0695, Train Accuracy: 42.97%
Epoch [1/100], Step [131/145], Loss: 1.0801, Train Accuracy: 38.28%
Epoch [1/100], Step [132/145], Loss: 1.0394, Train Accuracy: 42.97%
Epoch [1/100], Step [133/145], Loss: 1.0810, Train Accuracy: 37.50%
Epoch [1/100], Step [134/145], Loss: 1.0512, Train Accuracy: 42.97%
Epoch [1/100], Step [135/145], Loss: 1.0559, Train Accuracy: 38.28%
Epoch [1/100], Step [136/145], Loss: 1.0748, Train Accuracy: 34.38%
Epoch [1/100], Step [137/145], Loss: 1.0529, Train Accuracy: 46.09%
Epoch [1/100], Step [138/145], Loss: 1.0049, Tra

KeyboardInterrupt: 

In [25]:
total_step = len(test_loader)
print(total_step)
loss_list_test = []
acc_list_test = []
with torch.no_grad():
    for i, (signals, labels) in enumerate(test_loader):
        # Run the forward pass
        signals=signals
        labels=labels
        outputs = cnn(signals.double())
        loss = criterion(outputs, labels.long())
        loss_list_test.append(loss.item())
        if epoch%10 ==0:
            print(loss)
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels.long()).sum().item()
        acc_list_test.append(correct / total)
        if (epoch) % 1 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

1
Epoch [100/100], Step [1/1], Loss: 0.0686, Accuracy: 98.57%


In [26]:
# if you need to save
torch.save(cnn,'cnnTC3_fold3_45.pth')

/home/cmu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type CNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
